In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-domain-dataset/politifact_real.csv
/kaggle/input/multi-domain-dataset/politifact_fake.csv
/kaggle/input/multi-domain-dataset/gossipcop_real.csv
/kaggle/input/multi-domain-dataset/gossipcop_fake.csv


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
!pip uninstall -y wandb

In [4]:
!pip install --upgrade pip
!pip install virtualenv
!virtualenv kaggle_env
!source kaggle_env/bin/activate
!pip install torch==2.3.1 datasets==2.21.0 accelerate==0.33.0

created virtual environment CPython3.11.13.final.0-64 in 289ms
  creator CPython3Posix(dest=/kaggle/working/kaggle_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.9.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [5]:
# Uninstall the broken packages
!pip uninstall -y transformers
!pip uninstall -y tokenizers

# Remove leftover corrupted directories
!rm -rf /usr/local/lib/python3.11/dist-packages/~ransformers*
!rm -rf /usr/local/lib/python3.11/dist-packages/transformers*

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1


In [6]:
!pip install transformers==4.44.2

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [7]:
from transformers import Trainer,TrainingArguments
print("Transformers working fine!")

Transformers working fine!


In [8]:
!pip install protobuf==3.20.3 --force-reinstall

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-storage 2.19.0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-api-core 1.34.1 which is incompatible.
dataproc-spark-connect 0.7.5 requires google-api-core>=2.19, but you have google-api-core 1.34.1 w

In [9]:
!pip uninstall -y onnx tensorflow-metadata grpcio-status google-cloud-automl

In [10]:
!pip uninstall -y tensorflow-datasets tensorflow tensorflow-metadata onnx grpcio-status google-cloud-automl

In [7]:
import os,re,math
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
from transformers import Trainer,AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,EarlyStoppingCallback

In [8]:
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

In [9]:
true_politifact= "/kaggle/input/multi-domain-dataset/politifact_real.csv"
fake_politifact= "/kaggle/input/multi-domain-dataset/politifact_fake.csv"
true_gossipcop="/kaggle/input/multi-domain-dataset/gossipcop_real.csv"
fake_gossipcop="/kaggle/input/multi-domain-dataset/gossipcop_fake.csv"


# dataset 1
true_path_politifact=pd.read_csv(true_politifact)
fake_path_politifact=pd.read_csv(fake_politifact)

true_path_politifact["label"]=1
fake_path_politifact["label"]=0

POLITIFACT_CSV= pd.concat([true_path_politifact, fake_path_politifact], ignore_index=True)

POLITIFACT_CSV.head()



#dataset 2
true_path_gossipcop=pd.read_csv(true_gossipcop)
fake_path_gossipcop=pd.read_csv(fake_gossipcop)

true_path_gossipcop["label"]=1
fake_path_gossipcop["label"]=0

GOSSIPCOP_CSV= pd.concat([true_path_gossipcop, fake_path_gossipcop], ignore_index=True)

GOSSIPCOP_CSV.head()

,id,news_url,title,tweet_ids,label
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...,1
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...,1
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...,1
3,gossipcop-857248,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,868114761723936769\t868122567910936576\t868128...,1
4,gossipcop-884684,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,915528047004209152\t915529285171122176\t915530...,1


In [10]:
PROCESSED_DIR = "/kaggle/working/processed_datasets"
os.makedirs(PROCESSED_DIR, exist_ok=True)

politifact_path = os.path.join(PROCESSED_DIR, "politifact.csv")
POLITIFACT_CSV.to_csv(politifact_path, index=False)

gossipcop_path = os.path.join(PROCESSED_DIR, "gossipcop.csv")
GOSSIPCOP_CSV.to_csv(gossipcop_path, index=False)

POLITIFACT_CSV="/kaggle/working/processed_datasets/politifact.csv"
GOSSIPCOP_CSV="/kaggle/working/processed_datasets/gossipcop.csv"

In [11]:
MODEL_NAME = "distilbert-base-uncased"
MAX_LEN = 128            # titles: 64–128 is enough; smaller helps generalization
EPOCHS = 8               # early stopping will stop earlier
BATCH_TRAIN = 32         # increase/decrease based on GPU RAM
BATCH_EVAL  = 64
LR = 2e-5
WEIGHT_DECAY = 0.1
WARMUP_RATIO = 0.06
DROPOUT = 0.3            # stronger than default to reduce overfit
ATTN_DROPOUT = 0.3
VAL_FRAC = 0.1           # validation split from the training domain
TEST_FRAC = 0.2  

In [12]:
OUTPUT_DIR = "/kaggle/working/domain_transfer_distilbert"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [13]:
def clean_title(s: str) -> str:
    s = str(s)
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [14]:
def load_domain(path: str, domain_name: str) -> pd.DataFrame:
    """Load a domain CSV with columns ['title','label'].
       Returns cleaned, deduped DataFrame with ['title','label','domain'].
    """
    df = pd.read_csv(path)
    # Normalize column names if needed (uncomment if your CSV uses different names)
    # df = df.rename(columns={"headline": "title", "verdict": "label"})
    df = df[["title", "label"]].copy()
    df["title"] = df["title"].fillna("").map(clean_title)
    df = df[df["title"].str.len() > 3]  # drop empty/very short
    df["label"] = df["label"].astype(int)
    df["domain"] = domain_name
    df = df.drop_duplicates(subset=["title"]).reset_index(drop=True)
    return df

In [16]:
from datasets import Dataset

def to_hf_dataset(df: pd.DataFrame, tok, max_len: int) -> Dataset:
    ds = Dataset.from_pandas(df[["title", "label"]].reset_index(drop=True))
    def tok_fn(batch):
        return tok(batch["title"], padding="max_length", truncation=True, max_length=max_len)
    ds = ds.map(tok_fn, batched=True, desc="Tokenizing")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return ds

In [24]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def metrics_fn(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, zero_division=0),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
    }


In [18]:
class WeightedCELossTrainer(Trainer):
    """Inject class weights into CE loss."""
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        self.loss_fct = None

    def compute_loss(self, model, inputs, return_outputs=False):
        if self.loss_fct is None:
            if self.class_weights is not None:
                device = next(model.parameters()).device  
                cw = self.class_weights.to(device)
                self.loss_fct = nn.CrossEntropyLoss(weight=cw)
            else:
                self.loss_fct = nn.CrossEntropyLoss()

        labels = inputs.get("labels")
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=None)
        logits = outputs.logits
        loss = self.loss_fct(logits.view(-1, getattr(model, "module", model).config.num_labels
), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [22]:
from transformers import AutoConfig

def make_model_and_tok():
    tok=AutoTokenizer.from_pretrained(MODEL_NAME,use_fast=True)
    config=AutoConfig.from_pretrained(
        MODEL_NAME,
        num_labels=2,
        id2label={0:"FAKE",1:"REAL"},
        label2id={"FAKE":0,"REAL":1},
        dropout=DROPOUT,
        attention_dropout=ATTN_DROPOUT,
    )
    model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,config=config)
    return model,tok

def class_weights_from_labels(y:np.ndarray)->torch.Tensor:
    cw=compute_class_weight(class_weight="balanced",classes=np.array([0,1]),y=y)
    return torch.tensor(cw,dtype=torch.float)

In [20]:
def run_domain_transfer(train_domain_df: pd.DataFrame, test_domain_df: pd.DataFrame, tag: str):
    

    print(f"\n======== Experiment: {tag} ========")
    print(f"Train domain: {train_domain_df['domain'].iloc[0]} | Test domain: {test_domain_df['domain'].iloc[0]}")
    print("Train size (pre-split):", len(train_domain_df), "| Test (zero-shot) size:", len(test_domain_df))

    # In-domain split: create a train/val/test (holdout) from the training domain
    # First hold out TEST_FRAC for in-domain testing
    train_pool, in_domain_test = train_test_split(
        train_domain_df, test_size=TEST_FRAC, random_state=SEED, stratify=train_domain_df["label"]
    )
    # Then take a small validation from the remaining pool
    train_df, val_df = train_test_split(
        train_pool, test_size=VAL_FRAC, random_state=SEED, stratify=train_pool["label"]
    )

    # Safety checks: no title overlap
    assert set(train_df["title"]).isdisjoint(val_df["title"])
    assert set(train_df["title"]).isdisjoint(in_domain_test["title"])
    assert set(val_df["title"]).isdisjoint(in_domain_test["title"])

    # Build tokenizer & datasets
    model, tok = make_model_and_tok()
    train_ds = to_hf_dataset(train_df, tok, MAX_LEN)
    val_ds   = to_hf_dataset(val_df, tok, MAX_LEN)
    ind_test_ds = to_hf_dataset(in_domain_test, tok, MAX_LEN)
    xdomain_test_ds = to_hf_dataset(test_domain_df, tok, MAX_LEN)

    # Class weights from TRAIN ONLY
    class_weights = class_weights_from_labels(train_df["label"].values)

    # Training args
    out_dir = os.path.join(OUTPUT_DIR, tag)
    os.makedirs(out_dir, exist_ok=True)

    args = TrainingArguments(
        output_dir=out_dir,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        learning_rate=LR,
        per_device_train_batch_size=BATCH_TRAIN,
        per_device_eval_batch_size=BATCH_EVAL,
        num_train_epochs=EPOCHS,
        weight_decay=WEIGHT_DECAY,
        warmup_ratio=WARMUP_RATIO,
        max_grad_norm=1.0,
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        report_to="none",
        logging_steps=50,
        seed=SEED,
        dataloader_num_workers=2,
    )

    trainer = WeightedCELossTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tok,
        compute_metrics=metrics_fn,
        class_weights=class_weights,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    # Train
    trainer.train()

    # Evaluate in-domain holdout
    in_domain_metrics = trainer.evaluate(ind_test_ds)
    print("\nIn-domain holdout metrics:", in_domain_metrics)

    # Zero-shot on the other domain
    xdomain_metrics = trainer.evaluate(xdomain_test_ds)
    print("\nZero-shot cross-domain metrics:", xdomain_metrics)

    # Save
    trainer.save_model(out_dir)
    tok.save_pretrained(out_dir)
    print("Saved model to:", out_dir)

    return {
        "in_domain": in_domain_metrics,
        "cross_domain": xdomain_metrics,
        "save_dir": out_dir
    }

In [25]:
pol = load_domain(POLITIFACT_CSV, "politifact")
gossip = load_domain(GOSSIPCOP_CSV, "gossipcop")

print("PolitiFact size:", len(pol), "| GossipCop size:", len(gossip))
print("PolitiFact label balance:\n", pol["label"].value_counts(normalize=True))
print("GossipCop  label balance:\n", gossip["label"].value_counts(normalize=True))

# Run A) Train on PolitiFact -> Test on GossipCop
res_A = run_domain_transfer(train_domain_df=pol, test_domain_df=gossip, tag="train_POL_test_GOSSIP")

# Run B) Train on GossipCop -> Test on PolitiFact
res_B = run_domain_transfer(train_domain_df=gossip, test_domain_df=pol, tag="train_GOSSIP_test_POL")

print("\n==== Summary ====")
print("Train POL -> Test GOSSIP:", {k: {m: round(v, 4) for m, v in d.items() if isinstance(v, (int, float))} for k, d in res_A.items() if isinstance(d, dict)})
print("Train GOSSIP -> Test POL:", {k: {m: round(v, 4) for m, v in d.items() if isinstance(v, (int, float))} for k, d in res_B.items() if isinstance(d, dict)})

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PolitiFact size: 983 | GossipCop size: 20743
PolitiFact label balance:
 label
1    0.566633
0    0.433367
Name: proportion, dtype: float64
GossipCop  label balance:
 label
1    0.769754
0    0.230246
Name: proportion, dtype: float64

======== Experiment: train_POL_test_GOSSIP ========
Train domain: politifact | Test domain: gossipcop
Train size (pre-split): 983 | Test (zero-shot) size: 20743


Tokenizing:   0%|          | 0/707 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/79 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/197 [00:00<?, ? examples/s]

...

Tokenizing:   0%|          | 0/4149 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/983 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.595600,0.474482,0.760241,0.827107,0.929688,0.744914
2,0.466300,0.428486,0.814458,0.871560,0.933036,0.817684
3,0.419400,0.424480,0.799398,0.858479,0.939535,0.790297
4,0.387800,0.418133,0.832530,0.886159,0.929553,0.846635
5,0.357600,0.438533,0.837952,0.892011,0.915911,0.869327
6,0.334400,0.432910,0.831928,0.886815,0.920809,0.855243
7,0.313600,0.439361,0.831325,0.885901,0.924320,0.850548



In-domain holdout metrics: {'eval_loss': 0.43598473072052, 'eval_accuracy': 0.8353820197637986, 'eval_f1': 0.8897853800225917, 'eval_precision': 0.9180819180819181, 'eval_recall': 0.8631809643080777, 'eval_runtime': 9.2592, 'eval_samples_per_second': 448.097, 'eval_steps_per_second': 3.564, 'epoch': 7.0}

Zero-shot cross-domain metrics: {'eval_loss': 1.055993676185608, 'eval_accuracy': 0.3957273652085453, 'eval_f1': 0.35010940919037203, 'eval_precision': 0.4481792717086835, 'eval_recall': 0.2872531418312388, 'eval_runtime': 2.3366, 'eval_samples_per_second': 420.693, 'eval_steps_per_second': 3.424, 'epoch': 7.0}
Saved model to: /kaggle/working/domain_transfer_distilbert/train_GOSSIP_test_POL

==== Summary ====
Train POL -> Test GOSSIP: {'in_domain': {'eval_loss': 0.4004, 'eval_accuracy': 0.8376, 'eval_f1': 0.8644, 'eval_precision': 0.8226, 'eval_recall': 0.9107, 'eval_runtime': 0.6147, 'eval_samples_per_second': 320.478, 'eval_steps_per_second': 3.254, 'epoch': 8.0}, 'cross_domain': {

In [ ]:
import shutil

shutil.make_archive("/kaggle/working/model_POL", 'zip', "/kaggle/working/domain_transfer_distilbert/train_POL_test_GOSSIP")
shutil.make_archive("/kaggle/working/model_GOSSIP", 'zip', "/kaggle/working/domain_transfer_distilbert/train_GOSSIP_test_POL")
